# Table of Contents
* [Investigating programatically recovering hierarchy](#Investigating-programatically-recovering-hierarchy)
	* [Dataframe stuff](#Dataframe-stuff)
* [End](#End)


In [4]:
%%capture
from __future__ import division
import numpy as np
import pandas as pd
import scipy.stats as st
import itertools
import math
from collections import Counter, defaultdict, OrderedDict
%load_ext autoreload
%autoreload 2

In [2]:
import pickle
import json
import glob
import pprint

In [3]:
import pdfextraction.merge as me
import pdfextraction.unmerge as um
import pdfextraction.amt_boto_modules as amt
import pdfextraction.hierarchy_exctraction as hext

# Investigating programatically recovering hierarchy

## Setting up paths and other basics

In [ ]:
base_path = 'ai2-vision-turk-data/textbook-annotation-test/labeled-questions/'
test_path = 'ai2-vision-turk-data/textbook-annotation-test/hierarchy-ext-test/'

In [15]:
question_pages_df = pd.read_pickle('pages_w_questions.pkl')
question_pages = question_pages_df['page'].tolist()
to_review = pd.read_pickle('rev_seq.pkl').tolist()

book_breakdowns, page_ranges = amt.load_book_info()

verified_pages = to_review
verified_pages_minus_bad = [page for page in verified_pages if page not in bad_pages.tolist()]
print len(verified_pages)
print len(verified_pages_minus_bad)

856
468


## Basic structure

In [ ]:
# accuracy_total, errors_total, box_totals, ind_page_results = predict_and_verify_groups(single_page)
accuracy_total, errors_total, box_totals, ind_page_results = hext.predict_and_verify_groups(verified_pages_minus_bad, base_path)

print errors_total
print box_totals
print accuracy_total

for page_res in ind_page_results:
    hext.write_predicted_groups(page_res, base_path, test_path)

In [ ]:
problem_pages = [page[0] for page in ind_page_results if page[2] < 1.0]
problem_pagess = pd.Series(problem_pages)
# problem_pagess.to_pickle('problem_pages.pkl')

In [19]:
page_n = 0

In [281]:
cur_page = problem_pages[page_n:page_n + 1]
accuracy_total, errors_total, box_totals, ind_page_results = predict_and_verify_groups(cur_page)
page_n += 1
print ''
print cur_page
print '{} {}'.format('page', page_n)
print '{} {}'.format('number wrong=', errors_total)
print '{} {}'.format('out of', box_totals)
print '{} {}'.format('accuracy', accuracy_total)


actual 1
predicted 1
A. Write true or false.


actual 1
predicted 1
1. A conductor allows electric current to


actual 1
predicted 1
flow through it easily


actual 1
predicted 1
2. Electrical energy can make heat energy.


actual 1
predicted 1
3. Resistors convert less energy into heat


actual 1
predicted 1
than conductors do


actual 1
predicted 1
4. All metals conduct electricity


actual 1
predicted 1
equally we


actual 2
predicted 2
B. What is the main difference between a conductor and a resistor?


actual 2
predicted 3
C. How did resistors help the people who invented toasters?


actual 2
predicted 3
Explain your answer


[u'Daily_Science_Grade_4_Evan_Moor_131.jpeg']
page 9
number wrong= 2
out of 22
accuracy 0.909090909091


## Question-type specific structure

In [ ]:
# single_page = ['Daily_Science_Grade_2_Evan_Moor_33.jpeg']
single_page = ['Daily_Science_Grade_1_Evan_Moor_123.jpeg']
single_page = ['Daily_Science_Grade_1_Evan_Moor_93.jpeg']
# single_page = ['Daily_Science_Grade_1_Evan_Moor_44.jpeg'] # this is an example of the ocr serivce merging failing
# single_page = ['Daily_Science_Grade_2_Evan_Moor_107.jpeg'] # this is an example of unlabeled answer choices
# single_page = ['Daily_Science_Grade_2_Evan_Moor_12.jpeg']
# single_page = ['Daily_Science_Grade_2_Evan_Moor_123.jpeg'] 
# single_page = ['Daily_Science_Grade_2_Evan_Moor_144.jpeg'] #short answer and mc
# single_page = ['Daily_Science_Grade_2_Evan_Moor_146.jpeg'] # fib and mc
# single_page = ['Daily_Science_Grade_2_Evan_Moor_149.jpeg'] # another case where ocr service put on the same line 

1) question type hist
2) what cats turkers do well on 
3) total questions 

In [157]:
single_page

['Daily_Science_Grade_2_Evan_Moor_149.jpeg']

In [673]:
with open(test_path + single_page[0].replace('jpeg', 'json')) as f:
    ex_anno = json.load(f)

In [ ]:
ex_anno['parsed_questions'] = dict(complete_questions)
ex_anno['instructions'] = dict(instructions)

In [175]:
page_n = 32 

In [176]:
page_n += 1
print page_n
single_page = ['Daily_Science_Grade_2_Evan_Moor_' + str(page_n) +'.jpeg']

ctq = hext.parse_book(single_page, test_path)

for gn, group in ctq.items():
    print 'group ' + gn
    print
    for full_q, parts in group.items():
        if full_q == 'misfits':
            print 'misfits'
            for misfit, val in parts.items():
                for box, tv in val['asks'].items():
                    print '    ' + tv['contents']
        else:
            print 'question   ' + parts['category']  
            for ids, question_parts in parts['asks'].items():
                print question_parts['contents']
            for ids, ac in parts['answer_choices'].items():
                print '    '  + ac['possible_answer']['contents']
        print

33
group G2

question   True/False
4. A big tree always grows from a big seed.

question   True/False
5. Pollen helps plants make flowers

question   True/False
6. Mammals have hair and feed their babies mil

group G1

question   Multiple Choice
3. Plants have flowers so they can make
    B stems
    leaves (C) leaves
    A seeds

question   Multiple Choice
1. Which of these does NOT have a life cycle?
    pine tree
    OB OB
    CO mountain
    A butterfly

question   Multiple Choice
2. The first stage of the life cycle of a butterfly is
    B a Chrysalis
    C a caterpillar
    (A) an egg



# Dataframe stuff

In [118]:
# single_page_df = pd.DataFrame(vertically_ordered_question)

In [119]:
# single_page_df['rectangle'].iloc[0]

In [10]:
qa_df = pd.read_pickle('store_hit_results_metadata/question_anno/group_latest_combined/consensus_df.pkl')

In [11]:
bad_pages = pd.unique(qa_df.query('category == "No Consensus" or group_n == "No Consensus"')['page'])

In [12]:
consensus_only_df = qa_df[~qa_df['page'].isin(bad_pages)]

In [13]:
pd.unique(consensus_only_df['page']).shape

(468,)

In [ ]:
consensus_only_df.head(4)

In [179]:
bad_pages

array([u'Daily_Science_Grade_1_Evan_Moor_10.jpeg',
       u'Daily_Science_Grade_1_Evan_Moor_104.jpeg',
       u'Daily_Science_Grade_1_Evan_Moor_106.jpeg',
       u'Daily_Science_Grade_1_Evan_Moor_107.jpeg',
       u'Daily_Science_Grade_1_Evan_Moor_108.jpeg',
       u'Daily_Science_Grade_1_Evan_Moor_113.jpeg',
       u'Daily_Science_Grade_1_Evan_Moor_115.jpeg',
       u'Daily_Science_Grade_1_Evan_Moor_116.jpeg',
       u'Daily_Science_Grade_1_Evan_Moor_124.jpeg',
       u'Daily_Science_Grade_1_Evan_Moor_125.jpeg',
       u'Daily_Science_Grade_1_Evan_Moor_13.jpeg',
       u'Daily_Science_Grade_1_Evan_Moor_133.jpeg',
       u'Daily_Science_Grade_1_Evan_Moor_134.jpeg',
       u'Daily_Science_Grade_1_Evan_Moor_136.jpeg',
       u'Daily_Science_Grade_1_Evan_Moor_14.jpeg',
       u'Daily_Science_Grade_1_Evan_Moor_140.jpeg',
       u'Daily_Science_Grade_1_Evan_Moor_142.jpeg',
       u'Daily_Science_Grade_1_Evan_Moor_143.jpeg',
       u'Daily_Science_Grade_1_Evan_Moor_144.jpeg',
       u'Daily_

# End

In [363]:
# vertically_ordered_question = sorted(q_series.values(), key=lambda x: x['rectangle'][0][1])

# vertically_ordered_question_feat = [make_box_row(box, idx) for idx, box in enumerate(vertically_ordered_question)]

# boxes_w_predicts = assign_group_numbers(vertically_ordered_question_feat) 

In [310]:
t0 = 'Read each question. Fill in the bubble next to the correct answer.'
t1 = 'A distance'
t2 = 'OB balls'
t3 = 'Do push'
t4 = 'OD on a mountain'
t5 = 'CO Flowers might die out.'
re_pattern = re.compile('([A-Z])(?:[a-z]?){1}\s')

print re.findall(re_pattern, t0)
print re.findall(re_pattern, t1)
print re.findall(re_pattern, t2)
print re.findall(re_pattern, t3)
print re.findall(re_pattern, t4)
print re.findall(re_pattern, t5)

In [224]:
#print vertically_ordered_question
# qdf = pd.DataFrame(vertically_ordered_question)